# AOD logger

Generate a csv file with information about observations recorded on MeerKAT

Written by Sarah Buchner 2017

In [1]:
import pysolr
from IPython.core.display import display, HTML

import katdal
import katpoint

import os
import glob

import csv

archive = pysolr.Solr('http://192.168.1.50:8983/solr/kat_core')

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Setup variables

set up start and end date
optionally set up observer or instruction set

In [2]:
extra_search=[]
extra_components=[]

# uncomment these lines to add additional searches

#extra_components.append('InstructionSet:*beamform*')
#extra_components.append('Observer:*arah*')
#extra_search = ' AND '.join(extra_components)

startdate = '2018-03-04T00:00:00Z'
enddate = '2018-03-07T00:00:00Z'


outputfilenm="AODLog.csv"

# Get links to observation report

In [3]:
def get_obsreport_link(scheduleID):

    reduction = archive.search('CAS.ProductTypeName:MeerKATAR1ReductionProduct AND ExperimentID:%s' % scheduleID, sort="StartTime desc") 
    obsrep="None"
    if reduction.hits > 0:
        for r in reduction.docs:
            ReductionName=str(r["ReductionName"])
            if "Observation" in ReductionName:
                redfiles = r["CAS.ReferenceDatastore"] 
                for fl in redfiles:
                    if fl.endswith('html') or fl.endswith('pdf'):
                        link=fl[5:]
                        obsrep=link.replace('/var/kat','http://kat-archive.kat.ac.za')
    return obsrep

In [4]:
search_string=[]

search_components = []

if extra_search != []:
    search_components.append(extra_search)

search_components.append('CAS.ProductTypeName:MeerKATAR1TelescopeProduct')


search_components.append('StartTime:["%s" TO "%s"]'%(startdate,enddate))
search_string = ' AND '.join(search_components)


print "Searching archive for %s" %search_string
ar_res = archive.search( search_string ,rows=15000,**{'sort':'StartTime asc'})
print "Found %d hits" %ar_res.hits

ObsCount=ar_res.hits

Searching archive for CAS.ProductTypeName:MeerKATAR1TelescopeProduct AND StartTime:["2018-03-04T00:00:00Z" TO "2018-03-07T00:00:00Z"]
Found 29 hits


In [5]:
outputfile=open(outputfilenm,"w")

print "Writing observation log to %s" %(outputfilenm)


outputfile.write("SB;Observer;StartTime;description;duration;FreqChannels;DumpPeriod;NAnts;Ants;progress_report;archivelink;obsrep;QA1;VisFile;instructionSet\n")
                 
Count = 0
for r in ar_res.docs:

    Count = Count + 1
    # check for meta-data
    try:
        SB = r['ScheduleBlockIdCode']
        print "Processing %s: %d of %d" %(SB,Count,ObsCount)

    except:
        continue
    


    
    if r.has_key('Observer'):
        Observer = r['Observer']
    else:
        Observer = 'No observer'
    
    if r.has_key('Targets'):
        Target = r['Targets'][0]
    else:
        Target = 'No target'
        

    if r.has_key('InstructionSet'):
        instructions = r['InstructionSet']
    else:
        instructions = "No instructions"
        
    if r.has_key('Description'):
        description = r['Description']
    else:
        description = "No instructions"
    
    QA1=' '             
    if r.has_key('QA1'):
        QA1 = r['QA1']
    FreqChannels=r['NumFreqChannels']
    DumpPeriod=r['DumpPeriod']
    duration=r['Duration']
    Start=r['StartTime']
    #description=r['Description']
    Ants=' '.join(r['Antennas'])
    NAnts=len(r['Antennas'])
    VisFile=r['CAS.ReferenceDatastore'][0][5:]
    progress='http://obs.mkat.karoo.kat.ac.za:8081/tailtask/%s/progress'%(SB)
    archivelink = 'http://kat-archive.kat.ac.za:8080/archive_search/#{"ei":"%s","so":true,"a":true,"b":true,"ob":false,"rs":{}}'%(SB)
    obsrep=get_obsreport_link(SB)




    try:
        outputfile.write("%s;%s;%s;%s;" %(SB,Observer,Start,description))
                                      
        outputfile.write("%d;%d;%d;%d;%s;%s;" %(duration,FreqChannels,round(DumpPeriod),NAnts,Ants,progress))
        outputfile.write("%s;%s;%s;%s;" %(archivelink,obsrep,QA1,VisFile))

        outputfile.write("%s\n" %(instructions))
    except:
        print "issues with %s" %(SB)
    
    
  
outputfile.close()  
        

Writing observation log to AODLog.csv
Processing 20180303-0089: 1 of 29
Processing 20180304-0001: 2 of 29
Processing 20180304-0003: 3 of 29
Processing 20180304-0002: 4 of 29
Processing 20180304-0004: 5 of 29
Processing 20180304-0020: 6 of 29
Processing 20180304-0022: 7 of 29
Processing 20180304-0006: 8 of 29
Processing 20180304-0007: 9 of 29
Processing 20180305-0002: 10 of 29
Processing 20180305-0003: 11 of 29
Processing 20180305-0004: 12 of 29
Processing 20180305-0001: 13 of 29
Processing 20180304-0008: 14 of 29
Processing 20180303-0085: 15 of 29
Processing 20180305-0005: 16 of 29
Processing 20180304-0009: 17 of 29
Processing 20180304-0010: 18 of 29
Processing 20180304-0012: 19 of 29
Processing 20180304-0013: 20 of 29
Processing 20180304-0016: 21 of 29
Processing 20180304-0017: 22 of 29
Processing 20180305-0006: 23 of 29
Processing 20180305-0007: 24 of 29
Processing 20180304-0018: 25 of 29
Processing 20180305-0009: 26 of 29
Processing 20180305-0010: 27 of 29
Processing 20180305-0011: 